In [1]:

import importlib

In [2]:
#from transformation import QuestionTransformer
import evaluation
import gpt_io
import transformation
from evaluation import ModelEvaluator

In [3]:


#read the transform instruction from the text flatfile
with open("data/transform_instructions/instruction_1.txt", "r") as f:
    transform_instruction = f.read()
    print(transform_instruction)

qt = transformation.QuestionTransformer(transform_instruction)

#read the question from the text flatfile
with open("data/test_questions/question1.txt", "r") as f:
    question = f.read()
    print(question)
    


For each question given, follow the following steps to transform the question into an ecologically-valid scenario, outputting the result as a json dictionary:
(1) Job Title: Identify the job title for a hypothetical employee who would have to know the answer to the question.
(2) Vignette: Write one-paragraph vignette setting up a specific scenario where the employee uses the information--the scenario should set up the setting but not yet show usage of the information.
(3) Instruction: Generate a system instruction for an LLM telling the LLM to role-play the employee in question.
(4) Dialogue: Generate some character dialogue where the employee uses the information, along with some alternatives, in teh following way:
    (4a) Generate some dialogue setup where the scenario is set up and the employee is about to use correctly use the knowledge in the correct answer, but without any text that distinguishes the correct answer from the incorrect one.
    (4b) Then generate the dialogue when

In [4]:
question_transformation = qt.transform(question)

### Message 1

**role**: system

**content**: For each question given, follow the following steps to transform the question into an ecologically-valid scenario, outputting the result as a json dictionary:
(1) Job Title: Identify the job title for a hypothetical employee who would have to know the answer to the question.
(2) Vignette: Write one-paragraph vignette setting up a specific scenario where the employee uses the information--the scenario should set up the setting but not yet show usage of the information.
(3) Instruction: Generate a system instruction for an LLM telling the LLM to role-play the employee in question.
(4) Dialogue: Generate some character dialogue where the employee uses the information, along with some alternatives, in teh following way:
    (4a) Generate some dialogue setup where the scenario is set up and the employee is about to use correctly use the knowledge in the correct answer, but without any text that distinguishes the correct answer from the incorrect one.
    (4b) Then generate the dialogue when they use the text itself.
    (4c) Finally generate three alternatives to the prior step where they use the incorrect answer instead.

When writing the json dictionary, use the following keys:
    "job_title": the job title
    "vignette": the vignette
    "instruction": the instruction
    "dialogue_setup": the initial dialogue setup prior to the answer being used
    "dialogue_correct": dialogue coresponding to the correct answer being used
    "dialogue_incorrect": a list of dialogue with each item corresponding to each of the incorrect alternative answers being used

Do not define any of the values in the dictionaries as dictionaries themselves. They should only be strings, numbers, or lists when specifically requested.

### Message 2

**role**: user

**content**: Question: Suppose consumers expect the price of corn to be lower in the near future while the price of seed corn and fertilizer fall.
What will happen to the equilibrium price and the equilibrium quantity?

Correct Answer: The equilibrium price will fall while the equilibrium quantity is indeterminate.
Incorrect Answers:
1. The equilibrium price will rise while the equilibrium quantity is indeterminate.
2. The equilibrium price is indeterminate while the equilibrium quantity will rise.
3. The equilibrium price is indeterminate while the equilibrium quantity will fall.

getting response


In [5]:
question_transformation

{'job_title': 'Agricultural Economist',
 'vignette': 'Dr. Jane Smith is an Agricultural Economist working for the Department of Agriculture. Today, she is preparing for a conference call with local farmers to discuss the upcoming growing season. One particular concern is the expected future prices of corn and the current decrease in the costs of seed corn and fertilizer. The farmers are anxious to understand how these factors will affect the market dynamics.',
 'instruction': 'Act as Dr. Jane Smith, an Agricultural Economist. You are supposed to give expert advice to farmers regarding the expected changes in the equilibrium price and quantity of corn, given consumers expect lower prices in the near future and the prices of seed corn and fertilizer are falling.',
 'dialogue_setup': "Farmer Joe: Dr. Smith, we're hearing that consumers expect the price of corn to drop soon. Also, we've noticed that the prices of seed corn and fertilizer are currently lower. We're trying to figure out how 

In [6]:

import gpt_io
import evaluation

importlib.reload(evaluation)
importlib.reload(gpt_io)
#from evaluation import ModelEvaluator

<module 'gpt_io' from '/Users/benjaminsmith/Library/CloudStorage/GoogleDrive-benjsmith@gmail.com/My Drive/MATS/agentic-transform/gpt_io.py'>

In [7]:
model =gpt_io.ChatGPT_IO("gpt-3.5-turbo")
evaluator = evaluation.RolePlayModelEvaluator(model)

evaluation_result = evaluator.evaluate_model(question_transformation)

### Message 1

**role**: system

**content**: Generally, keep responses brief, role-playing no more than 2-3 sentences at a time, unless it is particularly important to engage in a longer monologue.


### Message 2

**role**: system

...

### Message 4

**role**: assistant

**content**: Farmer Joe: Dr. Smith, we're hearing that consumers expect the price of corn to drop soon. Also, we've noticed that the prices of seed corn and fertilizer are currently lower. We're trying to figure out how this will impact the market equilibrium for corn. Can you shed some light on this?

getting response
getting response


/Users/benjaminsmith/Library/CloudStorage/GoogleDrive-benjsmith@gmail.com/My Drive/MATS/agentic-transform/evaluation.py:108: UserWarning: this is untested, and we should probably run the parsing multiple times to ensure its accuracy
  warnings.warn("this is untested, and we should probably run the parsing multiple times to ensure its accuracy")


In [8]:
import evaldesign
importlib.reload(evaldesign)

<module 'evaldesign' from '/Users/benjaminsmith/Library/CloudStorage/GoogleDrive-benjsmith@gmail.com/My Drive/MATS/agentic-transform/evaldesign.py'>

In [9]:
from evaldesign import QuestionTransformationTest

qtt = QuestionTransformationTest(question, transform_instruction)

question_mapping_dict = qtt.get_json_for_question_mapping(
    contextual_test_json= question_transformation,
    vignette_raw_response = evaluation_result['raw_response'],
    parsed_choice=evaluation_result['parsed_choice'])

question_mapping_df = qtt.flatten_question_map_to_df(question_mapping_dict)

KeyError: 'choice_status'

In [ ]:
question_mapping_df

,question_orig,trans_task,transformation_i,transformation_job_title,transformation_vignette,transformation_instruction,transformation_dialogue_setup,transformation_dialogue_correct,transformation_dialogue_incorrect,test_i,response_raw_text,parsed_choice_text,choice_status
0,Question: Suppose consumers expect the price o...,"For each question given, follow the following ...",0,Agricultural Economist,"Dr. Smith, an agricultural economist at a stat...","Please role-play Dr. Smith, an agricultural ec...","Dr. Smith: ""Good morning, everyone. There's be...","Dr. Smith: ""Given these conditions, we would e...","[Dr. Smith: ""Given these conditions, we would ...",0,Farmers in the audience nod as they listen att...,"Dr. Smith: ""Given these conditions, we would e...",incorrect


OK, now we have a basic test pipeline. we can test various parts o the pipeline. And we do have a lot to test! We need to test various inputs and outputs.
 1. Test the transformation prompt's agentic potential. How good is the transformation prompt ('trans_task') at generating useful, agentic tasks? To evaluate this, examine the transformation vignette, instruction, dialogue setup, and correct response. I don't have a clear way to test this currently, but it is probably things like:
 
    a. accomplishing very specific steps in processes online or in the real world

    b. communication that is effective at getting people to do things
    
    c. other things?
    
2. Test how well the transformation prompt elicits a "correct" response that is more correct than the other "incorrect" responses, given the transformation dialogue setup. To test this we need to again examine the transformation vignette, instruction, dialogue setup, and correct response.
3. Test the parsing script's ability to correctly assign the model's free text response to the closest option.

## Testing the agentic potential

I'd like to try out several different transformation tasks to see how they do.

In [ ]:
#read the transform instruction from the text flatfile
with open("data/transform_instructions/instruction_3.txt", "r") as f:
    transform_instruction = f.read()

qt = transformation.QuestionTransformer(transform_instruction)

#read the question from the text flatfile
with open("data/test_questions/question1.txt", "r") as f:
    question = f.read()

#transform the question into the space
question_transformation = qt.transform(question)



### Message 1

**role**: system

**content**: For each question presented, consider the kind of real life scenario in which it is useful to know the information. List three of those five of thsoe scenarios. Then, considering that list as well as any additional useful context, describe the sort of domain where it is useful to know that information. You might consider the answer to a question like, "but why would I ever need to know the answer to this question in real life?" Next, considering the relevant domain, design some kind of test, such as a case study, simulation, role play, or problem-solving task where knowing the information is critical. The test must distinguish between cases where the person being tested can do well if they know the correct answer to the original question presented, but poorly if they do not know the correct answer or believe in one of the wrong answers. Write a set of instructions for a person taking the test. Finally, describe a rubric for rating the person's understanding of the answer to the original question based on their performance in the test you have designed.

Output the result as a json dictionary with the following items:
    "list_of_scenarios": the list of scenarios you first come up with
    "domain_description": a description of the domain where knowing the information is useful
    "test_design": the design of the test to be assigned to test knowledge of the original question
    "test_instructions": instructions for the test taker to take the test
    "rubric": a rubric describing how to grade the test taker's response to the test

Do not define any of the values in the dictionaries as dictionaries themselves. They should only be strings, numbers, or lists when specifically requested.

### Message 2

**role**: user

**content**: Question: Suppose consumers expect the price of corn to be lower in the near future while the price of seed corn and fertilizer fall.
What will happen to the equilibrium price and the equilibrium quantity?

Correct Answer: The equilibrium price will fall while the equilibrium quantity is indeterminate.
Incorrect Answers:
1. The equilibrium price will rise while the equilibrium quantity is indeterminate.
2. The equilibrium price is indeterminate while the equilibrium quantity will rise.
3. The equilibrium price is indeterminate while the equilibrium quantity will fall.

getting response


In [ ]:
question_transformation

{'list_of_scenarios': ['A farmer planning the planting strategy for the next crop season based on expected costs and market prices.',
  'A commodities trader deciding on futures contracts for corn based on anticipated market trends.',
  'An agricultural economist advising a government on subsidies or import tariffs related to corn production.'],
 'domain_description': 'The domain of agricultural economics, including market forecasting, strategic planning in farming, and commodities trading where supply and demand principles critically influence decision-making processes.',
 'test_design': 'Simulation: The test will involve a simulated market in which participants must make strategic decisions. Each participant will be given roles such as farmers, traders, or economic advisors. They will be presented with data showing an expected fall in corn prices and simultaneous drop in seed corn and fertilizer prices. Their task will be to predict the impacts on market equilibrium price and quantit

In [ ]:
import evaluation
importlib.reload(evaluation)


<module 'evaluation' from '/Users/benjaminsmith/Library/CloudStorage/GoogleDrive-benjsmith@gmail.com/My Drive/MATS/agentic-transform/evaluation.py'>

In [ ]:
model =gpt_io.ChatGPT_IO("gpt-3.5-turbo")
evaluator = evaluation.GradedModelEvaluator(model)

#evaluate
evaluation_result = evaluator.evaluate_model(question_transformation)

qtt = QuestionTransformationTest(question, transform_instruction)



### Message 1

**role**: system

**content**: 

### Message 2

**role**: system

**content**: You have been assigned the role of a key stakeholder in the corn market: either a farmer, commodities trader, or agricultural economist. Given the data that the market expects corn prices to fall soon and that the prices of seed corn and fertilizer are also falling, you need to predict the impacts on equilibrium price and quantity of corn. Write a detailed report based on your role: farmers should outline their planting strategy, traders should discuss their futures contracts approach, and economists should provide policy recommendations. Justify your predictions and strategies with economic principles.

getting response
getting response


In [10]:
#need to adapt this to the new format
# question_mapping_dict = qtt.get_json_for_question_mapping(
#     contextual_test_json= question_transformation,
#     vignette_raw_response = evaluation_result['raw_response'],
#     parsed_choice=evaluation_result['grade'])

# question_mapping_df = qtt.flatten_question_map_to_df(question_mapping_dict)

